### Exploring the WMS with OWSLib

In [2]:
!pip install owslib

  Running setup.py bdist_wheel for owslib: started
  Running setup.py bdist_wheel for owslib: finished with status 'error'
  Complete output from command C:\Users\PDudukin\Anaconda3\python.exe -u -c "import setuptools, tokenize;__file__='C:\\Users\\PDudukin\\AppData\\Local\\Temp\\pip-build-3co6jmwm\\owslib\\setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d C:\Users\PDudukin\AppData\Local\Temp\tmp0o30tbaipip-wheel- --python-tag cp35:
  usage: -c [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
     or: -c --help [cmd1 cmd2 ...]
     or: -c --help-commands
     or: -c cmd --help
  
  error: invalid command 'bdist_wheel'
  
  ----------------------------------------
  Running setup.py clean for owslib
Failed to build owslib
  Running setup.py install for owslib: started
    Running setup.py install for owslib: finished with status 'done'


  Failed building wheel for owslib


In [1]:
from owslib.wms import WebMapService


url = 'http://oos.soest.hawaii.edu/thredds/wms/hioos/satellite/dhw_5km'

web_map_services = WebMapService(url)

print('\n'.join(web_map_services.contents.keys()))

surface_flag
CRW_DHW
CRW_HOTSPOT
CRW_SST
CRW_SSTANOMALY


### Layer metadata

In [2]:
layer = 'CRW_SST'
wms = web_map_services.contents[layer]

name = wms.title

lon = (wms.boundingBox[0] + wms.boundingBox[2]) / 2.
lat = (wms.boundingBox[1] + wms.boundingBox[3]) / 2.
center = lat, lon

time_interval = '{0}/{1}'.format(
    wms.timepositions[0].strip(),
    wms.timepositions[-1].strip()
)
style = 'boxfill/sst_36'

if style not in wms.styles:
    style = None

### Single layer

In [11]:
import folium
from folium import plugins
from folium.plugins import timestamped_wmstilelayer

lon, lat = -50, -40

m = folium.Map(
    location=[lat, lon],
    zoom_start=5,
    control_scale=True
)

w = folium.WmsTileLayer(
    url=url,
    name=name,
    styles=style,
    fmt='image/png',
    transparent=True,
    layers=layer,
    overlay=True,
    COLORSCALERANGE='1.2,28',
)

w.add_to(m)

time = plugins.TimestampedWmsTileLayers(
    w,
    period='PT1H',
    time_interval=time_interval
)

time.add_to(m)

folium.LayerControl().add_to(m)

m

### Multiple layers

In [10]:
m = folium.Map(
    location=[lat, lon],
    zoom_start=5,
    control_scale=True
)

w0 = folium.WmsTileLayer(
    url=url,
    name='sea_surface_temperature',
    styles=style,
    fmt='image/png',
    transparent=True,
    layers='CRW_SST',
    overlay=True,
)

w1 = folium.WmsTileLayer(
    url=url,
    name='analysed sea surface temperature anomaly',
    styles=style,
    fmt='image/png',
    transparent=True,
    layers='CRW_SSTANOMALY',
    overlay=True,
)

w0.add_to(m)
w1.add_to(m)

time = folium.plugins.TimestampedWmsTileLayers(
    [w0, w1],
    period='PT1H',
    time_interval=time_interval
)

time.add_to(m)

folium.LayerControl().add_to(m)

m